In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn import metrics
from sklearn.decomposition import PCA
from scipy.stats import zscore
import matplotlib.pyplot as plt 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
parkinsons_data=pd.read_csv("D:\D_DATA\Learning\PGP-AIML\Ensemble Techniques/parkinsons.data",header=0)

print(parkinsons_data.shape)
print(parkinsons_data.head(10))
print(parkinsons_data.columns)
print(parkinsons_data.dtypes)
print(parkinsons_data.groupby(["status"]).count() )
print(parkinsons_data.isnull().sum().sum())
#print(parkinsons_data[~parkinsons_data.applymap(np.isreal).all(1)])

(195, 24)
             name  MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
0  phon_R01_S01_1      119.992       157.302        74.997         0.00784   
1  phon_R01_S01_2      122.400       148.650       113.819         0.00968   
2  phon_R01_S01_3      116.682       131.111       111.555         0.01050   
3  phon_R01_S01_4      116.676       137.871       111.366         0.00997   
4  phon_R01_S01_5      116.014       141.781       110.655         0.01284   
5  phon_R01_S01_6      120.552       131.162       113.787         0.00968   
6  phon_R01_S02_1      120.267       137.244       114.820         0.00333   
7  phon_R01_S02_2      107.332       113.840       104.315         0.00290   
8  phon_R01_S02_3       95.730       132.068        91.754         0.00551   
9  phon_R01_S02_4       95.056       120.103        91.226         0.00532   

   MDVP:Jitter(Abs)  MDVP:RAP  MDVP:PPQ  Jitter:DDP  MDVP:Shimmer    ...     \
0           0.00007   0.00370   0.00554     0.01109 

In [3]:
# Copy all the predictor variables into X dataframe and drop the dependent variable
parkinsons_data = parkinsons_data.drop('name', axis=1)
X = parkinsons_data.drop('status', axis=1)

y = parkinsons_data[['status']]

X.head()


,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.02971,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.04368,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.03590,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.03772,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.04465,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [4]:
# Split X and y into training and test set in 70:30 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)

In [5]:
#my_model=LogisticRegression()
DTEntropy_model=DecisionTreeClassifier(criterion='entropy',max_depth=5)


In [6]:
from sklearn.ensemble import BaggingClassifier
#Default base estimator is Decision Tree (CART)

bgcl = BaggingClassifier(base_estimator=DTEntropy_model,random_state=1,n_estimators=51)
bgcl.fit(X_train, y_train)
bgcl.score(X_test, y_test) # use this if you have reserved any dataset for testing

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.847457627118644

In [7]:
#In production any unseen new snapshot of data can be used for testing 
y_predict_DTEntropy = bgcl.predict(X_test)



In [8]:
#accuracy score
acc=metrics.accuracy_score(y_test,y_predict_DTEntropy)
print(acc)

0.847457627118644


In [9]:
cr=metrics.classification_report(y_test,y_predict_DTEntropy)
print(cr)

             precision    recall  f1-score   support

          0       0.92      0.58      0.71        19
          1       0.83      0.97      0.90        40

avg / total       0.86      0.85      0.84        59



In [10]:
cm=metrics.confusion_matrix(y_test,y_predict_DTEntropy)
print(cm)

[[11  8]
 [ 1 39]]


In [11]:
#print(DTEntropy_model.coef_)

pd.DataFrame(bgcl.predict_proba(X_test))

,0,1
0,0.039970,0.960030
1,0.000000,1.000000
2,0.019608,0.980392
3,0.513072,0.486928
4,0.039216,0.960784
5,0.313725,0.686275
6,0.000000,1.000000
7,0.156863,0.843137
8,0.677505,0.322495
9,0.103695,0.896305


In [12]:
#Perform Grid Search Method to find the optimal max_depth size
from sklearn.model_selection import GridSearchCV
parameter1={'max_depth':np.arange(1,10)}
parameter2={'min_samples_leaf':np.arange(1,10)}

GS1=GridSearchCV(DTEntropy_model,parameter1,refit=True,cv=3)
GS2=GridSearchCV(DTEntropy_model,parameter2,refit=True,cv=3)

print(GS1.fit(X,y))
print(GS2.fit(X,y))

#print(GS.best_params_)

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None

In [13]:
print("max_depth:best_params_:", GS1.best_params_)
print("max_depth:best_estimator_",GS1.best_estimator_)
print("max_depth:best_score_",GS1.best_score_)
print("max_depth:best_index_",GS1.best_index_)
print("max_depth:scorer_",GS1.scorer_)
print("max_depth:n_splits_",GS1.n_splits_)
#print(GS.refit_time_)
print("min_samples_leaf:best_params_:", GS2.best_params_)
print("min_samples_leaf:best_estimator_",GS2.best_estimator_)
print("min_samples_leaf:best_score_",GS2.best_score_)
print("min_samples_leaf:best_index_",GS2.best_index_)
print("min_samples_leaf:scorer_",GS2.scorer_)
print("min_samples_leaf:n_splits_",GS2.n_splits_)

#When left on its own, decision tree will continue to fit till each data point is in a different leaf. 
#This will not generalize well leading to overfitting. Hence we go for regularization through techniques like:
#(1) Limiting max depth of trees (2)bagging with more than 1 tree (3) stopping criterion like min gain, min samples in leaf, etc. 
#It seems there is too much of variation in sampling so i get different result. 
#Regularization: max_depth: The current result shows best score of .8308 with max_depth of 7. 
#Regularization: min_samples_leaf: The current result shows best score of 0.8462 with min_samples_leaf of 5. 


max_depth:best_params_: {'max_depth': 7}
max_depth:best_estimator_ DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
max_depth:best_score_ 0.8307692307692308
max_depth:best_index_ 6
max_depth:scorer_ <function _passthrough_scorer at 0x0000026331035730>
max_depth:n_splits_ 3
min_samples_leaf:best_params_: {'min_samples_leaf': 5}
min_samples_leaf:best_estimator_ DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            spli

In [14]:
pd.DataFrame(GS1.cv_results_)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000694,0.000981,0.0,0.0,1,{'max_depth': 1},0.753846,0.815385,0.753846,0.774359,0.029010,9,0.753846,0.884615,0.915385,0.851282,0.070033
1,0.000000,0.000000,0.0,0.0,2,{'max_depth': 2},0.846154,0.815385,0.753846,0.805128,0.038376,4,0.830769,0.884615,0.923077,0.879487,0.037859
2,0.005208,0.007365,0.0,0.0,3,{'max_depth': 3},0.784615,0.846154,0.769231,0.800000,0.033235,7,0.876923,0.946154,0.961538,0.928205,0.036802
3,0.000000,0.000000,0.0,0.0,4,{'max_depth': 4},0.830769,0.861538,0.753846,0.815385,0.045291,2,0.923077,0.992308,1.000000,0.971795,0.034592
4,0.000000,0.000000,0.0,0.0,5,{'max_depth': 5},0.830769,0.876923,0.707692,0.805128,0.071428,4,0.976923,1.000000,1.000000,0.992308,0.010879
5,0.005207,0.007364,0.0,0.0,6,{'max_depth': 6},0.830769,0.861538,0.738462,0.810256,0.052298,3,0.976923,1.000000,1.000000,0.992308,0.010879
6,0.000000,0.000000,0.0,0.0,7,{'max_depth': 7},0.830769,0.892308,0.769231,0.830769,0.050246,1,1.000000,1.000000,1.000000,1.000000,0.000000
7,0.000000,0.000000,0.0,0.0,8,{'max_depth': 8},0.830769,0.784615,0.769231,0.794872,0.026149,8,1.000000,1.000000,1.000000,1.000000,0.000000
8,0.000000,0.000000,0.0,0.0,9,{'max_depth': 9},0.830769,0.800000,0.784615,0.805128,0.019188,4,1.000000,1.000000,1.000000,1.000000,0.000000


In [15]:
pd.DataFrame(GS2.cv_results_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.000000,0.000000,0.000000,0.000000,1,{'min_samples_leaf': 1},0.830769,0.830769,0.738462,0.800000,0.043514,4,0.976923,1.000000,1.000000,0.992308,0.010879
1,0.007280,0.006057,0.000000,0.000000,2,{'min_samples_leaf': 2},0.830769,0.861538,0.723077,0.805128,0.059363,3,0.969231,0.984615,0.976923,0.976923,0.006281
2,0.002644,0.000489,0.000000,0.000000,3,{'min_samples_leaf': 3},0.830769,0.830769,0.738462,0.800000,0.043514,4,0.969231,0.984615,0.969231,0.974359,0.007252
3,0.001325,0.000937,0.000332,0.000470,4,{'min_samples_leaf': 4},0.830769,0.830769,0.738462,0.800000,0.043514,4,0.969231,0.976923,0.953846,0.966667,0.009594
4,0.000000,0.000000,0.000000,0.000000,5,{'min_samples_leaf': 5},0.830769,0.907692,0.800000,0.846154,0.045291,1,0.969231,0.969231,0.953846,0.964103,0.007252
5,0.000000,0.000000,0.005209,0.007367,6,{'min_samples_leaf': 6},0.830769,0.907692,0.800000,0.846154,0.045291,1,0.969231,0.961538,0.953846,0.961538,0.006281
6,0.005217,0.007377,0.000000,0.000000,7,{'min_samples_leaf': 7},0.830769,0.753846,0.723077,0.769231,0.045291,8,0.961538,0.915385,0.953846,0.943590,0.020190
7,0.000000,0.000000,0.000000,0.000000,8,{'min_samples_leaf': 8},0.800000,0.753846,0.769231,0.774359,0.019188,7,0.946154,0.915385,0.953846,0.938462,0.016617
8,0.005207,0.007364,0.000000,0.000000,9,{'min_samples_leaf': 9},0.800000,0.753846,0.723077,0.758974,0.031612,9,0.938462,0.915385,0.953846,0.935897,0.015806


In [16]:
Rf_model = RandomForestClassifier(n_estimators=50,criterion='entropy',max_depth=5,random_state=0 )

In [17]:
param_grid = [{'n_estimators': range(2, 500, 30), 'max_features': [2, 4, 6, 8]}]
GS3 = GridSearchCV(Rf_model, param_grid,cv=5)

print(GS3.fit(X,y))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y wa

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': range(2, 500, 30), 'max_features': [2, 4, 6, 8]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [18]:
print("max_depth:best_params_:", GS3.best_params_)
print("max_depth:best_estimator_",GS3.best_estimator_)
print("max_depth:best_score_",GS3.best_score_)
print("max_depth:best_index_",GS3.best_index_)
print("max_depth:scorer_",GS3.scorer_)
print("max_depth:n_splits_",GS3.n_splits_)
#Optimal number of tress that gives best result is 32. 


max_depth:best_params_: {'max_features': 8, 'n_estimators': 32}
max_depth:best_estimator_ RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features=8, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=32, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
max_depth:best_score_ 0.8153846153846154
max_depth:best_index_ 52
max_depth:scorer_ <function _passthrough_scorer at 0x0000026331035730>
max_depth:n_splits_ 5


In [19]:
pd.DataFrame(GS3.cv_results_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\util

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.006919,0.005880,0.000612,0.000500,2,2,"{'max_features': 2, 'n_estimators': 2}",0.850,0.600,0.692308,...,0.692308,0.118566,68,0.825806,0.916129,0.916667,0.974522,0.949045,0.916434,0.050301
1,0.033863,0.006897,0.000800,0.001600,2,32,"{'max_features': 2, 'n_estimators': 32}",0.800,0.825,0.871795,...,0.800000,0.046914,19,0.993548,0.993548,1.000000,1.000000,0.980892,0.993598,0.006977
2,0.056645,0.010077,0.005325,0.005516,2,62,"{'max_features': 2, 'n_estimators': 62}",0.750,0.850,0.871795,...,0.789744,0.061448,56,1.000000,0.980645,1.000000,1.000000,0.980892,0.992307,0.009422
3,0.077570,0.004617,0.001601,0.001961,2,92,"{'max_features': 2, 'n_estimators': 92}",0.750,0.850,0.871795,...,0.789744,0.061448,56,1.000000,1.000000,0.993590,1.000000,0.993631,0.997444,0.003130
4,0.111071,0.011115,0.003234,0.003065,2,122,"{'max_features': 2, 'n_estimators': 122}",0.775,0.825,0.871795,...,0.794872,0.047988,47,1.000000,1.000000,0.993590,1.000000,0.974522,0.993622,0.009867
5,0.131946,0.006168,0.005413,0.002798,2,152,"{'max_features': 2, 'n_estimators': 152}",0.750,0.825,0.923077,...,0.800000,0.068647,19,1.000000,1.000000,1.000000,1.000000,0.980892,0.996178,0.007643
6,0.149280,0.014008,0.011624,0.003653,2,182,"{'max_features': 2, 'n_estimators': 182}",0.775,0.825,0.897436,...,0.800000,0.056542,19,1.000000,1.000000,1.000000,1.000000,0.993631,0.998726,0.002548
7,0.178584,0.011512,0.006343,0.005900,2,212,"{'max_features': 2, 'n_estimators': 212}",0.775,0.825,0.897436,...,0.800000,0.056542,19,0.993548,1.000000,1.000000,1.000000,0.980892,0.994888,0.007431
8,0.211000,0.014930,0.015088,0.003594,2,242,"{'max_features': 2, 'n_estimators': 242}",0.750,0.825,0.871795,...,0.789744,0.051074,56,0.993548,1.000000,1.000000,1.000000,0.987261,0.996162,0.005104
9,0.224084,0.006311,0.010293,0.005919,2,272,"{'max_features': 2, 'n_estimators': 272}",0.775,0.825,0.871795,...,0.794872,0.047988,47,0.993548,1.000000,1.000000,1.000000,0.993631,0.997436,0.003141


In [20]:
pd.DataFrame(GS3.grid_scores_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:762: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


,parameters,mean_validation_score,cv_validation_scores
0,"{'max_features': 2, 'n_estimators': 2}",0.692308,"[0.85, 0.6, 0.6923076923076923, 0.526315789473..."
1,"{'max_features': 2, 'n_estimators': 32}",0.800000,"[0.8, 0.825, 0.8717948717948718, 0.73684210526..."
2,"{'max_features': 2, 'n_estimators': 62}",0.789744,"[0.75, 0.85, 0.8717948717948718, 0.71052631578..."
3,"{'max_features': 2, 'n_estimators': 92}",0.789744,"[0.75, 0.85, 0.8717948717948718, 0.71052631578..."
4,"{'max_features': 2, 'n_estimators': 122}",0.794872,"[0.775, 0.825, 0.8717948717948718, 0.736842105..."
5,"{'max_features': 2, 'n_estimators': 152}",0.800000,"[0.75, 0.825, 0.9230769230769231, 0.7368421052..."
6,"{'max_features': 2, 'n_estimators': 182}",0.800000,"[0.775, 0.825, 0.8974358974358975, 0.736842105..."
7,"{'max_features': 2, 'n_estimators': 212}",0.800000,"[0.775, 0.825, 0.8974358974358975, 0.736842105..."
8,"{'max_features': 2, 'n_estimators': 242}",0.789744,"[0.75, 0.825, 0.8717948717948718, 0.7368421052..."
9,"{'max_features': 2, 'n_estimators': 272}",0.794872,"[0.775, 0.825, 0.8717948717948718, 0.736842105..."
